In [1]:
from datasets import load_dataset

from elasticsearch import Elasticsearch, helpers

# Connect to ES instance
es = Elasticsearch(hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)

C:\Users\sumeet\AppData\Local\Temp\ipykernel_24904\1325504120.py:6: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=["http://localhost:9200"], timeout=60, retry_on_timeout=True)


In [ ]:
!pip install elasticsearch

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT

In [ ]:
server = Popen(['C:/apps/elasticsearch-8.4.3/bin/elasticsearch.bat'], stdout=PIPE, stderr=STDOUT)
!sleep 30

In [ ]:
from datasets import load_dataset

from elasticsearch import Elasticsearch, helpers

# Connect to ES instance
es = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)

# Load HF dataset
dataset = load_dataset("ag_news", split="train")["text"][:50000]

# Elasticsearch bulk buffer
buffer = []
rows = 0

for x, text in enumerate(dataset):
  # Article record
  article = {"_id": x, "_index": "articles", "title": text}

  # Buffer article
  buffer.append(article)

  # Increment number of articles processed
  rows += 1

  # Bulk load every 1000 records
  if rows % 1000 == 0:
    helpers.bulk(es, buffer)
    buffer = []

    print("Inserted {} articles".format(rows), end="\r")

if buffer:
  helpers.bulk(es, buffer)

print("Total articles inserted: {}".format(rows))

In [ ]:

import csv
from os.path import abspath, join, dirname, exists
import tqdm
import urllib3
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk


NYC_RESTAURANTS = (
    "https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD"
)
DATASET_PATH = join("C:\\apps\\", "nyc-restaurants.csv")
CHUNK_SIZE = 16384


def download_dataset():
    """Downloads the public dataset if not locally downlaoded
    and returns the number of rows are in the .csv file.
    """
    if not exists(DATASET_PATH):
        http = urllib3.PoolManager()
        resp = http.request("GET", NYC_RESTAURANTS, preload_content=False)

        if resp.status != 200:
            raise RuntimeError("Could not download dataset")

        with open(DATASET_PATH, mode="wb", encoding="utf8") as f:
            chunk = resp.read(CHUNK_SIZE)
            while chunk:
                f.write(chunk)
                chunk = resp.read(CHUNK_SIZE)

    with open(DATASET_PATH, encoding="utf8") as f:
        return sum([1 for _ in f]) - 1


def create_index(client):
    """Creates an index in Elasticsearch if one isn't already there."""
    client.indices.create(
        index="nyc-restaurants",
        body={
            "settings": {"number_of_shards": 1},
            "mappings": {
                "properties": {
                    "name": {"type": "text"},
                    "borough": {"type": "keyword"},
                    "cuisine": {"type": "keyword"},
                    "grade": {"type": "keyword"},
                    "location": {"type": "geo_point"},
                }
            },
        },
        ignore=400,
    )


def generate_actions():
    """Reads the file through csv.DictReader() and for each row
    yields a single document. This function is passed into the bulk()
    helper to create many documents in sequence.
    """
    with open(DATASET_PATH, mode="r", encoding="utf8") as f:
        reader = csv.DictReader(f)

        for row in reader:
            doc = {
                "_id": row["CAMIS"],
                "name": row["DBA"],
                "borough": row["BORO"],
                "cuisine": row["CUISINE DESCRIPTION"],
                "grade": row["GRADE"] or None,
            }

            lat = row["Latitude"]
            lon = row["Longitude"]
            if lat not in ("", "0") and lon not in ("", "0"):
                doc["location"] = {"lat": float(lat), "lon": float(lon)}
            yield doc


def main():
    print("Loading dataset...")
    number_of_docs = download_dataset()

    client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)
    print("Creating an index...")
    create_index(client)

    print("Indexing documents...")
    progress = tqdm.tqdm(unit="docs", total=number_of_docs)
    successes = 0
    for ok, action in streaming_bulk(
        client=client, index="nyc-restaurants", actions=generate_actions(),
    ):
        progress.update(1)
        successes += ok
    print("Indexed %d/%d documents" % (successes, number_of_docs))


if __name__ == "__main__":
    main()

In [ ]:
client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)

In [ ]:
a = client.search(index='nyc-restaurants', body={
  'query': {
    'match': {
      '_id': '50127304',
     }
  }
})
a

In [15]:
import pandas as pd
import numpy as np
import json
import os
import uuid


from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
pd.set_option('display.notebook_repr_html', False)
tqdm.pandas()

In [16]:
df  = pd.read_csv("job_posts.csv")
df.head(10)

                                             jobpost          date  \
0  AMERIA Investment Consulting Company\r\nJOB TI...   Jan 5, 2004   
1  International Research & Exchanges Board (IREX...   Jan 7, 2004   
2  Caucasus Environmental NGO Network (CENN)\r\nJ...   Jan 7, 2004   
3  Manoff Group\r\nJOB TITLE:  BCC Specialist\r\n...   Jan 7, 2004   
4  Yerevan Brandy Company\r\nJOB TITLE:  Software...  Jan 10, 2004   
5  Boutique "Appollo"\r\nJOB TITLE:  Saleswoman\r...  Jan 10, 2004   
6  OSI Assistance Foundation - Armenian Branch Of...  Jan 11, 2004   
7  International Research & Exchanges Board (IREX...  Jan 13, 2004   
8  Yerevan Brandy Company \r\nJOB TITLE:  Assista...  Jan 13, 2004   
9  American Embassy Yerevan\r\nANNOUNCEMENT NUMBE...  Jan 13, 2004   

                                               Title  \
0                            Chief Financial Officer   
1  Full-time Community Connections Intern (paid i...   
2                                Country Coordinator   
3    

In [17]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

In [18]:
token_instance = Tokenizer()

In [19]:
df = df.head(5000)
df = df.dropna(how='all')

In [20]:
df['vector'] = df['jobpost'].progress_apply(token_instance.get_token)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
PUT posting 
{
   "settings":{
      "index":{
         "number_of_shards":1,
         "number_of_replicas":0,
         "knn":{
            "algo_param":{
               "ef_search":40,
               "ef_construction":40,
               "m":"4"
            }
         }
      },
      "knn":true
   },
   "mappings":{
      "properties":{
         "vector":{
            "type":"knn_vector",
            "dimension":384
         },
         "title":{
            "type":"text",
            "fields":{
               "keyword":{
                  "type":"keyword",
                   "index":false
               }
            }
         },
         "company":{
            "type":"keyword",
            "index":false
         },
         "location":{
            "type":"keyword",
            "index":false
         },
         "salary":{
            "type":"keyword",
            "index":false
         },
         "job_description":{
            "type":"keyword",
            "index":false
         }
      }
   }
}

In [ ]:
elk_data = df.to_dict("records")

In [ ]:
es = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)

In [ ]:
for x in elk_data:
    try:
        _={
           "title":x.get("Title", ""),
            "company":x.get("Company", ""),
            "location":x.get("Location", ""),
            "salary":x.get("Salary", ""),
            "vector":x.get("vector", ""),
            "job_description":x.get("JobDescription", ""),

        }
        es.index(index='posting',body=_)
    except Exception as e:pass
    

In [ ]:
INPUT = input("Enter the Input Query ")
token_vector = token_instance.get_token(INPUT)

query  ={
  "_source": ["title", "job_description"], 
   "size":50,
   "query":{
      "bool":{
         "must":[
            {
               "knn":{
                  "vector":{
                     "vector":token_vector,
                     "k":20
                  }
               }
            }
         ]
      }
   }
}


res = es.search(index='posting1',
                size=50,
                body=query,
                request_timeout=55)

title = [x['_source']  for x in res['hits']['hits']] 
title

## FInal Code Starts from here

In [33]:
from tqdm.notebook import tqdm
import os
import json
import pandas as pd
import time
import numpy as np
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk

In [8]:
dataset = pd.read_csv("D:\\arxiv dump\\Embeddings/sample_100.csv")

def convert(item):
    item = item.strip()  # remove spaces at the end
    item = item[1:-1]    # remove `[ ]`
    item = np.fromstring(item, sep=' ')  # convert string to `numpy.array`
    return item

dataset['vector'] = dataset['embeddings'].apply(convert)

In [9]:
dataset.head(10)

,Unnamed: 0,abstract,embeddings,vector
0,0,A fully differential calculation in perturba...,[-1.33643940e-01 7.99820647e-02 -2.59015679e-...,"[-0.13364394, 0.0799820647, -0.259015679, -0.0..."
1,1,"We describe a new algorithm, the $(k,\ell)$-...",[ 1.40859067e-01 2.30508655e-01 1.20633028e-...,"[0.140859067, 0.230508655, 0.120633028, 0.0760..."
2,2,The evolution of Earth-Moon system is descri...,[ 8.67411792e-02 1.22752758e-02 -6.91849440e-...,"[0.0867411792, 0.0122752758, -0.069184944, 0.0..."
3,3,We show that a determinant of Stirling cycle...,[ 2.25209355e-01 6.65704012e-02 -5.51288947e-...,"[0.225209355, 0.0665704012, -0.000551288947, 0..."
4,4,In this paper we show how to compute the $\L...,[-4.51628491e-03 1.31734580e-01 -2.85967231e-...,"[-0.00451628491, 0.13173458, -0.285967231, -0...."
5,5,We study the two-particle wave function of p...,[ 3.04478467e-01 -1.56092525e-01 -2.15403959e-...,"[0.304478467, -0.156092525, -0.215403959, -0.0..."
6,6,A rather non-standard quantum representation...,[ 1.98111072e-01 -4.66411188e-03 4.31902222e-...,"[0.198111072, -0.00466411188, 0.0431902222, 0...."
7,7,A general formulation was developed to repre...,[ 1.01819381e-01 1.36552304e-01 1.39253110e-...,"[0.101819381, 0.136552304, 0.13925311, 0.10627..."
8,8,We discuss the results from the combined IRA...,[ 5.14137596e-02 -2.31310248e-01 4.08286601e-...,"[0.0514137596, -0.231310248, 0.0408286601, -0...."
9,9,Partial cubes are isometric subgraphs of hyp...,[ 5.11773705e-01 -9.93338078e-02 8.86382014e-...,"[0.511773705, -0.0993338078, 0.0886382014, -0...."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  101 non-null    int64 
 1   abstract    101 non-null    object
 2   embeddings  101 non-null    object
 3   vector      101 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [3]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)

C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\938878863.py:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)


In [4]:
from transformers import AutoTokenizer, TFAutoModel

# model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_ckpt = "sentence-transformers/all-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)


def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

import torch
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="tf")
    encoded_input = {k: v for k, v in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    # return  mean_pooling(model_output, encoded_input["attention_mask"])
    return cls_pooling(model_output)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
embeddings = get_embeddings([dataset.iloc[0]['abstract']])
print(embeddings.shape)

(1, 384)


In [22]:
for index, row in dataset.iterrows():
     print(f'Index: {index}, abstract: {row.get("abstract", 0)}')
    

Index: 0, abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
that enhanced sensitivity to the signal can be obtained with judicious
selection of event

In [77]:
def create_index(client):
    client.indices.create(
        index="ssd-search",
        body={
            "settings": {"number_of_shards": 1},
            "mappings": {
                "dynamic": "true",
                "_source": {
                    "enabled": "true"
                },
                "properties": {
                    "id": {
                        "type": "keyword",
                        "index":"false"
                    },
                    "abstract": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword", 
                                "index":"false"
                            }
                        }
                    },
                    "vector": {
                        "type": "dense_vector",
                        "dims": 384
                    }
                }
            }
},
        ignore=400,
    )
    
    
    
    


In [88]:
def generate_actions(dataset):
    def convert(item):
        item = item.strip()  # remove spaces at the end
        item = item[1:-1]    # remove `[ ]`
        item = np.fromstring(item, sep=' ')  # convert string to `numpy.array`
        return item

    dataset['embeddings'] = dataset['embeddings'].apply(convert)

    for index, row in dataset.iterrows():
            doc = {
                "id": index,
                "abstract": row.get("abstract", 0),
                "embeddings": row.get("embeddings", 0),
            }
            yield doc


def main():
    print("Loading dataset...")
    dataset = pd.read_csv("D:\\arxiv dump\\Embeddings/Test-0.csv")
    number_of_docs = len(dataset) + 1

    client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)
    print("Creating an index...")
    create_index(client)

    print("Indexing documents...")
    progress = tqdm(unit="docs", total=number_of_docs)
    successes = 0
    for ok, action in streaming_bulk(
        client=client, index="ssd-search", actions=generate_actions(dataset),
    ):
        progress.update(1)
        successes += ok
    print("Indexed %d/%d documents" % (successes, number_of_docs))


if __name__ == "__main__":
    main()

Loading dataset...


C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\1011029156.py:24: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)


Creating an index...


C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\3881900672.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(
C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\3881900672.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(
C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Indexing documents...


  0%|          | 0/100001 [00:00<?, ?docs/s]

C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn

Indexed 100000/100001 documents


## Do not run below

In [56]:
action_list = []
client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)
for index, row in dataset.iterrows():

            doc = {
                "id": index,
                "abstract": row.get("abstract", 0),
                "vector": row.get("vector", 0),
                 "_index": "ssd-search"
            }

            action_list.append(doc)
helpers.bulk(client, action_list)
es.indices.delete(index='ssd-search', ignore=[400, 404])

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\300644945.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  client = Elasticsearch(['https://localhost:9200'], http_auth=('elastic', 'P_YhQNcMjc36xSELpgzJ'), verify_certs=False)


In [90]:
es.indices.delete(index='ssd-search', ignore=[400, 404])

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\384970040.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='ssd-search', ignore=[400, 404])
C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True})

In [89]:
embedding_start = time.time()
question = "Deep learning contextual EMbeddings"
question_embedding = get_embeddings([question]).numpy()

embedding_time = time.time() - embedding_start

script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, 'vector' ) + 1.0",
            "params": {"query_vector": question_embedding[0]}
        }
    }
}


client.search(
        index="ssd-search",
        body={
            "size": 10,
            "query": script_query,
            "_source": {"includes": ["id", "abstract"]}
        }
    )

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\1609754686.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.search(
C:\Users\sumeet\anaconda3\envs\Optimization-Python\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'took': 693, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.69564, 'hits': [{'_index': 'ssd-search', '_id': '-1hxD4QBnvXQquafFwUZ', '_score': 1.69564, '_source': {'id': 24666, 'abstract': "  We propose a range of deep lexical acquisition methods which make use of\nmorphological, syntactic and ontological language resources to model word\nsimilarity and bootstrap from a seed lexicon. The different methods are\ndeployed in learning lexical items for a precision grammar, and shown to each\nhave strengths and weaknesses over different word classes. A particular focus\nof this paper is the relative accessibility of different language resource\ntypes, and predicted ``bang for the buck'' associated with each in deep lexical\nacquisition applications.\n"}}, {'_index': 'ssd-search', '_id': 'EVhxD4QBnvXQquaf9dLJ', '_score': 1.6925199, '_source': {'id': 76912, 'abstra

In [91]:
dataset = pd.read_csv("D:\\arxiv dump\\Embeddings/Test-1.csv")
dataset.head(2)

,Unnamed: 0,abstract,embeddings
0,100000,No-scale supersymmetry or gaugino mediation ...,[-4.60246354e-02 4.32418548e-02 -8.72858316e-...
1,100001,Numerous authors have referred to room-tempe...,[ 2.32891068e-01 -3.49794663e-02 1.66781843e-...


## Final code ends here

In [92]:
import numpy as np
import pandas as pd
import gc
import os
import json
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import re
import csv  
year_pattern = r'([1-2][0-9]{3})'
pd.set_option('display.max_colwidth', None)


In [93]:
df = pd.read_csv('D:\\arxiv dump\\processed.csv')
df.head(1)

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5160\143463918.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\arxiv dump\\processed.csv')


,Unnamed: 0,id,submitter,title,categories,abstract,update_date,authors_parsed,cat-short,categories_map,new_categories
0,0,704.0001,Pavel Nadolsky,Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies,hep-ph,"A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n",2008-11-26,"[['Balázs', 'C.', ''], ['Berger', 'E. L.', ''], ['Nadolsky', 'P. M.', ''], ['Yuan', 'C. -P.', '']]",hep-ph,High Energy Physics - Phenomenology,['High Energy Physics - Phenomenology']
